In [ ]:
import py21cmcast as p21c
import numpy as np 

from astropy import units

%load_ext autoreload
%autoreload 2

## I. Create the configuration files for varying parameters

See the examples of master configuration files in the config folder. Running $\tt init\_fisher\_from\_fiducial$ will create a folder (named after the run name set in the master configuration file) and sub configuration files inside (which correspond to the variation of each parameter). Once they are created each can by run with the script $\tt run\_fisher.py$. Note that the script takes at least one argument (the name of the sub configuration files to run) and three optional: the number of threads, the number of the first seed, and the number of times the same script should be run with increasing seeds. The latter is only usefull to run several fiducial cases in order to determine what the 'average Universe' looks like. In practice, for a Fisher forecast one should fix the seed to a single value for all the runs. 

In [ ]:
configuration_file = "../config/Fiducial1.config"
p21c.init_runs_from_fiducial(configuration_file, clean_existing_dir=False)

## II. Define the grid of modes and redshifts

Let us call the function ${\tt define\_grid\_modes\_redshits()}$. It returns the bin edges for a fixed grid of modes and redshifts.

- The redshift bins are computed from the frequency according to the formula
\begin{equation}
z_n \equiv \frac{f_{21}}{\frac{f_{21}}{1+z_0} - n B} -1
\end{equation}
where $B$ is the bandwidth and $f_{21}$ is the 21cm frequency.

- The mode bins are computed from the bandwidth according to the formula
\begin{equation}
k_n \equiv {\rm max} \left\{k_{\rm min},  \delta k \right\} + n \delta k \quad {\rm with} \quad \delta k \equiv 2\pi \frac{f_{21}}{B} \frac{H(z_0)}{c(1+z_0)^2}
\end{equation}
This definition corresponds to the bins set by ${\tt 21cmSense}$. Note that the choice of redshift $z_0$ fixes the step of the mode bins.

In [ ]:
z_bins, k_bins = p21c.define_grid_modes_redshifts(6., 8 * units.MHz, z_max = 22, k_min = 0.1 / units.Mpc, k_max = 1 / units.Mpc)
print("The redshift bin edges are:")
print(z_bins)
print("The mode bin edges are:")
print(k_bins)

## III. Define the fiducial model and get its power spectrum

A ${\tt Fiducial}$ object can be defined by specifying the path where the lightcones are saved (depends on what was set on the config file) as well as  the redshifts and modes bin edges. Note that we must set by hand whether the mode bins are linearly of logarithmically spaced with the option ${\tt logk}$  that is by default ${\tt False}$ as the output mode bin deges returned by ${\tt define\_grid\_modes\_redshits()}$ are linearly spaced. Further options can be specified, such as an ${\tt observation}$  and a fraction of modeling error ${\tt frac\_mod}$ (see below).

In [ ]:
fiducial = p21c.Fiducial("../runs/FID1", z_bins, k_bins, False)

Once the fiducial model is defined, we can check some of its properties:
- by plotting the neutral hydrogen fraction 'xH_box' (provided they have been asked as output of the lightcone) and computing the optical depth to reionization
- by computing the reduced $\chi^2$ value from the UV luminosity data

In [ ]:
fiducial.plot_xH_box()
fiducial.plot_global_signal()

chi2 = fiducial.chi2_UV_luminosity_functions(plot = True)

print('The optical depth to reionization is:', fiducial.tau_ion)
print('The reduced chi^2 from UV luminosity data is:', chi2)

## IV. Evaluate experimental noise

The experimental noise is derived using $\tt 21cmSense$. To that end one simply has to fix the 'observation' attribute of the fiducial object. The only valid input for now is 'HERA' but other can be added straightforwardly.

In [ ]:
%%capture 
## Remove the long 21cmSense output

fiducial.observation = "HERA"
fiducial.plot_power_spectrum()

## V. Define the Parameter objects

(This part is the longest and can take minutes or hours depending on the box resolution). Here all power spectra and associated derivatives are pre-computed and stored as atrribute of the Parameter object. With the load and save options of the Parameter class, if this part of code has been executed once, then all the objects can be reloaded in an instant (since every pre-computed quantity is stored in .npz and .pkl files).

In [ ]:
#parameter_names = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC', 'M_TURN', 't_STAR', 'L_X', 'NU_X_THRESH']
#parameter_names = ['F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI',  't_STAR', 'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'L_X_MINI', 
#                   'DM_LOG10_LIFETIME', 'DM_FHEAT_APPROX_PARAM_LOG10_F0', 'DM_FHEAT_APPROX_PARAM_A', 'DM_FHEAT_APPROX_PARAM_B', 
#                   'LOG10_XION_at_Z_HEAT_MAX', 'LOG10_TK_at_Z_HEAT_MAX']
#parameter_names = ['F_STAR10', 'ALPHA_STAR', 't_STAR', 'F_ESC10', 'ALPHA_ESC', 'L_X', 
#                   'DM_LOG10_LIFETIME', 'DM_FHEAT_APPROX_PARAM_LOG10_F0', 'DM_FHEAT_APPROX_PARAM_A', 'DM_FHEAT_APPROX_PARAM_B', 
#                   'LOG10_XION_at_Z_HEAT_MAX', 'LOG10_TK_at_Z_HEAT_MAX']

parameter_names = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC', 't_STAR', 'L_X', 'DM_LOG10_LIFETIME', 'DM_LOG10_MASS']

params = [None] * len(parameter_names)

for iname, name in enumerate(parameter_names) :  
    params[iname] = p21c.Parameter(fiducial=fiducial, name=name, verbose = False)
    params[iname].plot_power_spectra(color=['b', 'k', 'r'])

## VI. Evaluate the Fisher matrix and its inverse

From the list of parameters objects defined above one can directly compute the Fisher matrix with the $\tt evaluate\_fisher\_matrix$ function. One can add a modeling noise to the power spectrum with the attribute $\tt frac\_noise$ of the fiducial (as a percentage of the fiducial value). The triangle plot figure can be produced with the function $\tt make\_triangle\_plot$ and plotted. Finally, the matrices can be displayed nicely with the $\tt display\_matrix$ function.

In [ ]:
fiducial.frac_noise = 0
fisher_matrix     = p21c.evaluate_fisher_matrix(params)
covariance_matrix = np.linalg.inv(fisher_matrix['matrix'])

In [ ]:
fig = p21c.make_triangle_plot(covariance_matrix, fisher_matrix['name'], fiducial.astro_params)
fig.savefig(fiducial.dir_path + '/triangle_plot.pdf')

In [ ]:
p21c.display_matrix(fisher_matrix['matrix'], fisher_matrix['name'])
print("-------")
p21c.display_matrix(covariance_matrix, fisher_matrix['name'])